In [1]:
import pandas as pd
import requests
import time

from etl_resources import sqlite_connection, get_api_key, get_symbol_list

In [2]:
def build_statements_dataset():
    
    '''
    This function uses the alphavantage api to pull financial statements for 
    S&P 100 equities. Balance sheets, cash flow, income statements, and earnings
    estimates are included.
    '''
    
    api_key = get_api_key()
    symbol_list = get_symbol_list()
    con = sqlite_connection()
    
    statements = {'INCOME_STATEMENT':'quarterlyReports',
                  'BALANCE_SHEET':'quarterlyReports',
                  'CASH_FLOW':'quarterlyReports'}
    
    for statement, filters in statements.items(): 
        
        for symbol in symbol_list:

            time.sleep(20) # Handle the rate limiter
            url = f"https://www.alphavantage.co/query?function={statement}&symbol={symbol}&apikey={api_key}"

            try:
                
                print(f"Parsing {statement} for {symbol}")
                response = requests.get(url)
                json = response.json()[f"{filters}"]
                df = pd.DataFrame.from_dict(json)
                df['ticker'] = symbol

                table_name = statement.lower() 
                df.to_sql(name=table_name, if_exists='append', index=False, con=con)

            except:
                # TODO: Better failure logging
                print(f"Failed: {url}")
 


In [ ]:
build_statements_dataset()